<a href="https://colab.research.google.com/github/Noble-Lab/casanovo_asms2023/blob/main/Casanovo_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

* Use `pip` to easily install the latest version of Casanovo.

In [1]:
!pip install casanovo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.1/235.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

* Use `pip` to easily install the latest version of Casanovo.After installation, test that it was successful by viewing the Casanovo command line interface help:

In [2]:
!casanovo --help

2023-05-31 07:12:55.490140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 07:12:56.352900: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Usage: casanovo [OPTIONS]

  Casanovo: De novo mass spectrometry peptide sequencing with a transformer model.

  Yilmaz, M., Fondrie, W. E., Bittremieux, W., Oh,
  S. & Noble, W. S. De novo mass spectrometry
  peptide sequencing with a transformer model.
  Proceedings of the 39th International Conference
  on Machine Learning - ICML '22 (2022)
  doi:10.1101/2022.02.07.479481.

  Official code website: https://github.com/Noble-
  Lab/casanovo

Options:
  --mode [denovo|train|eval]  The mode in which to run Casanovo:
                              - "denovo" wi

## Sequence new mass spectra


* Download a sample MGF file with 128 MS/MS spectra

In [17]:
!wget https://noble.gs.washington.edu/~melih/one_psm.mgf

--2023-05-31 17:56:54--  https://noble.gs.washington.edu/~melih/one_psm.mgf
Resolving noble.gs.washington.edu (noble.gs.washington.edu)... 128.95.55.224
Connecting to noble.gs.washington.edu (noble.gs.washington.edu)|128.95.55.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1044 (1.0K)
Saving to: ‘one_psm.mgf’

one_psm.mgf         100%[===================>]   1.02K  --.-KB/s    in 0s      

2023-05-31 17:56:54 (52.1 MB/s) - ‘one_psm.mgf’ saved [1044/1044]



* To sequence your own mass spectra with Casanovo, use the denovo mode. 
* Just provide the spectra file (in mzML, mzXML, or MGF format) and path for peptide predictions in the mzTab format.

In [18]:
!casanovo --mode=denovo --peak_path=one_psm.mgf --output=sample_output.mztab

# Don't worry about the printed warnings, we will resolve/suppress those in future releases.

2023-05-31 17:57:10.680116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 17:57:11.537895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Global seed set to 454
2023-05-31 17:57:12,601 INFO [casanovo/MainProcess] casanovo._get_model_weights : Model weights file /root/.cache/casanovo/casanovo_massivekb_v3_0_0.ckpt retrieved from local cache
2023-05-31 17:57:12,601 INFO [casanovo/MainProcess] casanovo.main : Casanovo version 3.3.0
2023-05-31 17:57:12,602 DEBUG [casanovo/MainProcess] casanovo.main : mode = denovo
2023-05-31 17:57:12,602 DEBUG [casanovo/MainProcess] casanovo.main : model = /root/.cache/casanovo/casanovo_massivekb_v3_0_0.ckpt
2023-05-31 17:57:12,602 DEBUG [casanovo/MainProcess] cas

* You will see a log file and the mzTab output file created in the directory where Casanovo was run.
* mzTab files list predicted peptides,their identifiers and confidence scores in tab-delimited format.

In [21]:
! cat sample_output.mztab

MTD	mzTab-version	1.0.0
MTD	mzTab-mode	Summary
MTD	mzTab-type	Identification
MTD	description	Casanovo identification file sample_output
MTD	software[1]	[MS, MS:1003281, Casanovo, 3.3.0]
MTD	psm_search_engine_score[1]	[MS, MS:1001143, search engine specific score for PSMs, ]
MTD	fixed_mod[1]	[UNIMOD, UNIMOD:4, Carbamidomethyl, ]
MTD	fixed_mod[1]-site	C
MTD	variable_mod[1]	[UNIMOD, UNIMOD:7, Deamidated, ]
MTD	variable_mod[1]-site	N
MTD	variable_mod[2]	[UNIMOD, UNIMOD:7, Deamidated, ]
MTD	variable_mod[2]-site	Q
MTD	variable_mod[3]	[UNIMOD, UNIMOD:35, Oxidation, ]
MTD	variable_mod[3]-site	M
MTD	variable_mod[4]	[UNIMOD, UNIMOD:385, Ammonia-loss, ]
MTD	variable_mod[4]-site	N-term
MTD	variable_mod[5]	[UNIMOD, UNIMOD:5, Carbamyl, ]
MTD	variable_mod[5]-site	N-term
MTD	variable_mod[6]	[UNIMOD, UNIMOD:1, Acetyl, ]
MTD	variable_mod[6]-site	N-term
MTD	software[1]-setting[1]	model = /root/.cache/casanovo/casanovo_massivekb_v3_0_0.ckpt
MTD	software[1]-setting[2]	config_filename = default
MTD	software

* There are two other options you may want to add when sequencing new spectra with Casanovo:
>* 1 - Use a different set of pre-trained model weights, e.g. `--model=/path/to/pretrained/weights.ckpt`
>* 2 - Use a different configuration, e.g. `--config=/path/to/config.yaml`

* A sample command would look like:
> `casanovo --mode=denovo --model=casanovo_massivekb.ckpt --peak_path=one_psm.mgf --config=custom_config.yaml --output=one_psm_output.mztab`

***PRO TIP:*** You can sequence spectra from multiple input files by indicating 
 `--peak_path=*.mgf`

## Train a new model

* To train a model from scratch, run:


In [ ]:
#FIXME: training currently doesn't work due to unreleased fix to a validation step bug
!casanovo --mode=train --peak_path=one_psm.mgf  --peak_path_val=one_psm.mgf --config=custom_config.yaml

* Ground truth peptide labels must be provided for training and validation spectra, e.g. SEQ field in an MGF file.
* In the config file, you can set the path to model weights chekpointed during training

* You can also fine-tune a previously trained model with your own data, specify the starting model weights using `--model` and set `train_from_scratch: False` in the config file.

* A sample command would look like:
> `casanovo --mode=train --model=casanovo_massivekb.ckpt --peak_path=one_psm.mgf  --peak_path_val=one_psm.mgf --config=custom_config.yaml`